In [1]:
from pycocotools.coco import COCO

In [2]:
import os
import nltk
import pickle
import numpy as np
from PIL import Image
from collections import Counter
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
 
    

import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms
import torchvision.models as models
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pack_padded_sequence

C:\Users\rpals\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\rpals\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rpals\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
class Vocab(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.w2i = {}
        self.i2w = {}
        self.index = 0
 
    def __call__(self, token):
        if not token in self.w2i:
            return self.w2i['<unk>']
        return self.w2i[token]
 
    def __len__(self):
        return len(self.w2i)
    
    def add_token(self, token):
        if not token in self.w2i:
            self.w2i[token] = self.index
            self.i2w[self.index] = token
            self.index += 1
            
def build_vocabulary(json, threshold):
    """Build a simple vocabulary wrapper."""
    coco = COCO(json)
    counter = Counter()
    ids = coco.anns.keys()
    for i, id in enumerate(ids):
        caption = str(coco.anns[id]['caption'])
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        counter.update(tokens)
 
        if (i+1) % 50000 == 0:
            print("[{}/{}] Tokenized the captions.".format(i+1, len(ids)))
 
    # If the word frequency is less than 'threshold', then the word is discarded.
    tokens = [token for token, cnt in counter.items() if cnt >= threshold]
 
    # Create a vocab wrapper and add some special tokens.
    vocab = Vocab()
    vocab.add_token('<pad>')
    vocab.add_token('<start>')
    vocab.add_token('<end>')
    vocab.add_token('<unk>')
 
    # Add the words to the vocabulary.
    for i, token in enumerate(tokens):
        vocab.add_token(token)
    return vocab
 
vocab = build_vocabulary(json='coco2014/annotations/captions_train2014.json', threshold=4)
vocab_path = './coco2014/vocabulary.pkl'
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)
print("Total vocabulary size: {}".format(len(vocab)))
print("Saved the vocabulary wrapper to '{}'".format(vocab_path))


loading annotations into memory...
Done (t=0.95s)
creating index...
index created!
[50000/414113] Tokenized the captions.
[100000/414113] Tokenized the captions.
[150000/414113] Tokenized the captions.
[200000/414113] Tokenized the captions.
[250000/414113] Tokenized the captions.
[300000/414113] Tokenized the captions.
[350000/414113] Tokenized the captions.
[400000/414113] Tokenized the captions.
Total vocabulary size: 9956
Saved the vocabulary wrapper to './coco2014/vocabulary.pkl'


In [16]:
vocab.__len__()

9956

In [17]:
def reshape_image(image, shape):
    """Resize an image to the given shape."""
    return image.resize(shape, Image.ANTIALIAS)
 
def reshape_images(image_path, output_path, shape):
    """Reshape the images in 'image_path' and save into 'output_path'."""
    if not os.path.exists(output_path):
        os.makedirs(output_path)
 
    images = os.listdir(image_path)
    num_im = len(images)
    for i, im in enumerate(images):
        with open(os.path.join(image_path, im), 'r+b') as f:
            with Image.open(f) as image:
                image = reshape_image(image, shape)
                image.save(os.path.join(output_path, im), image.format)
        if (i+1) % 100 == 0:
            print ("[{}/{}] Resized the images and saved into '{}'."
                   .format(i+1, num_im, output_path))

image_path = './coco2014/train2014/'
output_path = './coco2014/resized_images/'
image_shape = [256, 256]
reshape_images(image_path, output_path, image_shape)

[100/82783] Resized the images and saved into './coco2014/resized_images/'.
[200/82783] Resized the images and saved into './coco2014/resized_images/'.
[300/82783] Resized the images and saved into './coco2014/resized_images/'.
[400/82783] Resized the images and saved into './coco2014/resized_images/'.
[500/82783] Resized the images and saved into './coco2014/resized_images/'.
[600/82783] Resized the images and saved into './coco2014/resized_images/'.
[700/82783] Resized the images and saved into './coco2014/resized_images/'.
[800/82783] Resized the images and saved into './coco2014/resized_images/'.
[900/82783] Resized the images and saved into './coco2014/resized_images/'.
[1000/82783] Resized the images and saved into './coco2014/resized_images/'.
[1100/82783] Resized the images and saved into './coco2014/resized_images/'.
[1200/82783] Resized the images and saved into './coco2014/resized_images/'.
[1300/82783] Resized the images and saved into './coco2014/resized_images/'.
[1400/82

[21200/82783] Resized the images and saved into './coco2014/resized_images/'.
[21300/82783] Resized the images and saved into './coco2014/resized_images/'.
[21400/82783] Resized the images and saved into './coco2014/resized_images/'.
[21500/82783] Resized the images and saved into './coco2014/resized_images/'.
[21600/82783] Resized the images and saved into './coco2014/resized_images/'.
[21700/82783] Resized the images and saved into './coco2014/resized_images/'.
[21800/82783] Resized the images and saved into './coco2014/resized_images/'.
[21900/82783] Resized the images and saved into './coco2014/resized_images/'.
[22000/82783] Resized the images and saved into './coco2014/resized_images/'.
[22100/82783] Resized the images and saved into './coco2014/resized_images/'.
[22200/82783] Resized the images and saved into './coco2014/resized_images/'.
[22300/82783] Resized the images and saved into './coco2014/resized_images/'.
[22400/82783] Resized the images and saved into './coco2014/resi

OSError: image file is truncated (9 bytes not processed)